## Custom Task Finetuning with Zephyr 7B Quantized Model for Customer Support Chatbot


#### Installing Necessary Dependencies

- LLM: Zephyr 7B Alpha

Description: Zephyr-7B-α, part of the Zephyr language model series, is fine-tuned from mistralai/Mistral-7B-v0.1. It excels in English language processing, thanks to Direct Preference Optimization (DPO).

- Model Type: 7B-parameter GPT-like model.

- License: MIT

- TRL Library:

 A comprehensive toolset for transformer language models and diffusion models training, with reinforcement learning support. It's built on top of the Hugging Face transformers library.

- PeFT Library

Description: PEFT efficiently adapts pre-trained language models for downstream tasks by fine-tuning a minimal set of additional parameters.

- Accelerate

Description: Simplifies training and inference scaling with just a few lines of code, compatible with any distributed configuration.

- BitsAndBytes

Description: A lightweight wrapper for CUDA custom functions, including 8-bit optimizers, matrix multiplication (LLM.int8()), and quantization functions.

- AutoGPTQ

Description: A user-friendly LLMs quantization package based on the GPTQ algorithm.

- Optimum

Description: An extension of Transformers with optimization tools for efficient training and deployment on specific hardware configurations.


In [ ]:
! pip install -qU \
       transformers \
       datasets \
       trl \
       peft \
       accelerate \
       bitsandbytes \
       auto-gptq \
       optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 36.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1

#### Login to HuggingFace-Hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

#### Importing Necessary Packages

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer

#### Model Congiguration

In [ ]:
MODEL_ID = "TheBloke/zephyr-7B-alpha-GPTQ"
DATASET_ID = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"
CONTEXT_FIELD= ""
INSTRUCTION_FIELD = "instruction"
TARGET_FIELD = "response"
BITS = 4
DISABLE_EXLLAMA = True
DEVICE_MAP = "auto"
USE_CACHE = False
LORA_R = 16
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
BIAS = "none"
TARGET_MODULES = ["q_proj", "v_proj"]
TASK_TYPE = "CAUSAL_LM"
OUTPUT_DIR = "zephyr-support-chatbot"
BATCH_SIZE = 8
GRAD_ACCUMULATION_STEPS = 1
OPTIMIZER = "paged_adamw_32bit"
LR = 2e-4
LR_SCHEDULER = "cosine"
LOGGING_STEPS = 50
SAVE_STRATEGY = "epoch"
NUM_TRAIN_EPOCHS = 1
MAX_STEPS = 250
FP16 = True
PUSH_TO_HUB = True
DATASET_TEXT_FIELD = "text"
MAX_SEQ_LENGTH = 1024
PACKING = False

#### Loading Dataset

In [ ]:
data = load_dataset(DATASET_ID,split='train')
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 26872
})

In [ ]:
df = data.to_pandas()
df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [ ]:
def process_data_sample(example):
        '''
        Helper function to process the dataset sample by adding prompt and clean if necessary.

        Args:
        example: Data sample

        Returns:
        processed_example: Data sample post processing
        '''
        processed_example = "<|system|>\n You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.\n<|user|>\n" + example[INSTRUCTION_FIELD] + "\n<|assistant|>\n" + example[TARGET_FIELD]
        return processed_example

#### Preprocess Datatset

In [ ]:
df[DATASET_TEXT_FIELD] = df[[INSTRUCTION_FIELD, TARGET_FIELD]].apply(lambda x: process_data_sample(x), axis=1)
df.head()
print(df.iloc[0]['text'])

<|system|>
 You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.
<|user|>
question about cancelling order {{Order Number}}
<|assistant|>
I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.


In [ ]:
# Convert the Dataframe to Huggingface Dataset
processed_data = Dataset.from_pandas(df[[DATASET_TEXT_FIELD]])
processed_data

Dataset({
    features: ['text'],
    num_rows: 26872
})

In [ ]:
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("TheBloke/zephyr-7B-alpha-GPTQ")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Model Preparation for Fine-Tuning: Quantization and LoRa Module Integration with Bitsandbytes Configuration
bnb_config = GPTQConfig(bits=4,
                        disable_exllama=True,
                        device_map="auto",
                        use_cache=False,
                        lora_r=16,
                        lora_alpha=16,
                        tokenizer=tokenizer
                                )
#
model = AutoModelForCausalLM.from_pretrained("TheBloke/zephyr-7B-alpha-GPTQ",
                                              quantization_config=bnb_config,
                                              device_map="auto",
                                              use_cache=False,
                                              )

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [ ]:
print("\n====================================================================\n")
print("\t\t\tDOWNLOADED MODEL")
print(model)
print("\n====================================================================\n")



			DOWNLOADED MODEL
MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLUActivation()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)




In [ ]:
# Update Model Configurations
model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
print("\n====================================================================\n")
print("\t\t\tMODEL CONFIG UPDATED")
print("\n====================================================================\n")

peft_config = LoraConfig(
                            r=LORA_R,
                            lora_alpha=LORA_ALPHA,
                            lora_dropout=LORA_DROPOUT,
                            bias=BIAS,
                            task_type=TASK_TYPE,
                            target_modules=TARGET_MODULES
                        )

model = get_peft_model(model, peft_config)
print("\n====================================================================\n")
print("\t\t\tPREPARED MODEL FOR FINETUNING")
print(model)
print("\n====================================================================\n")



			MODEL CONFIG UPDATED




			PREPARED MODEL FOR FINETUNING
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (rotary_emb): MistralRotaryEmbedding()
              (k_proj): QuantLinear()
              (o_proj): QuantLinear()
              (q_proj): QuantLinear(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embeddi

In [ ]:
# Configuration of Training Loop Parameters within the TrainingArguments Class
training_arguments = TrainingArguments(
                                        output_dir=OUTPUT_DIR,
                                        per_device_train_batch_size=BATCH_SIZE,
                                        gradient_accumulation_steps=GRAD_ACCUMULATION_STEPS,
                                        optim=OPTIMIZER,
                                        learning_rate=LR,
                                        lr_scheduler_type=LR_SCHEDULER,
                                        save_strategy=SAVE_STRATEGY,
                                        logging_steps=LOGGING_STEPS,
                                        num_train_epochs=NUM_TRAIN_EPOCHS,
                                        max_steps=MAX_STEPS,
                                        fp16=FP16,
                                        push_to_hub=PUSH_TO_HUB)

In [ ]:
# Training and Uploading: Model Training and Hub Deployment
print("\n====================================================================\n")
print("\t\t\tPREPARED FOR FINETUNING")
print("\n====================================================================\n")

trainer = SFTTrainer(
                        model=model,
                        train_dataset=processed_data,
                        peft_config=peft_config,
                        dataset_text_field=DATASET_TEXT_FIELD,
                        args=training_arguments,
                        tokenizer=tokenizer,
                        packing=PACKING,
                        max_seq_length=MAX_SEQ_LENGTH
                    )
trainer.train()

print("\n====================================================================\n")
print("\t\t\tFINETUNING COMPLETED")
print("\n====================================================================\n")

trainer.push_to_hub()



			PREPARED FOR FINETUNING




Map:   0%|          | 0/26872 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:214: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the diff

Step,Training Loss
50,0.948600
100,0.682400
150,0.637600
200,0.605200




			FINETUNING COMPLETED




'https://huggingface.co/kaifahmad/zephyr-support-chatbot/tree/main/'

In [ ]:
# Inference with Fine-Tuned Model
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch

def process_data_sample(example):

    processed_example = "<|system|>\n You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.\n<|user|>\n" + example["instruction"] + "\n<|assistant|>\n"

    return processed_example

tokenizer = AutoTokenizer.from_pretrained("/content/zephyr-support-chatbot")

inp_str = process_data_sample(
    {
        "instruction": "i have a question about cancelling order {{Order Number}}",
    }
)

inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/zephyr-support-chatbot",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
# Measure and print the time taken for model inference.
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

<|system|>
 You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.
<|user|>
i have a question about cancelling order {{Order Number}}
<|assistant|>
I'm on it! I understand that you have a question about canceling order {{Order Number}}. Rest assured, I'm here to assist you with that. To cancel your order, you can reach out to our customer support team at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website URL}}. They will be able to guide you through the cancellation process and provide you with any necessary information. If you have any specific concerns or need further assistance, please don't hesitate to let me know. I'm here to help you every step of the way!
10.018016576766968


In [ ]:
# Tokenize and process data for the given instruction.
inp_str = process_data_sample(
    {
        "instruction": "i have a question about the delay in order {{Order Number}}",
    }
)

inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")

In [ ]:
# Generate and print model output, and measure time for inference.
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

<|system|>
 You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.
<|user|>
i have a question about the delay in order {{Order Number}}
<|assistant|>
I'm on the same page that you have a question about the delay in order {{Order Number}}. I apologize for any inconvenience this may have caused. To provide you with the most accurate information, could you please provide me with the specific details of your order? This will help me investigate the issue and provide you with a resolution. Your satisfaction is our top priority, and we are committed to resolving this matter promptly. Thank you for bringing this to our attention. Your cooperation is greatly appreciated. Let's work together to find a solution.
13.959253072738647
